<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install LAMDA-SSL
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendige bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import sklearn.model_selection
from statistics import mean

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

## Egendefinerte moduler

In [5]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [6]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag

In [7]:
def evaluering_beregning(y_test, y_test_prediksjon, y_trening, y_trening_prediksjon):
  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)

  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [8]:
def evaluering_resultater(detailed_objective):
  rmse_test = detailed_objective(study.best_trial)[0]
  r2_test = detailed_objective(study.best_trial)[1]
  mae_test = detailed_objective(study.best_trial)[2]
  mape_test = detailed_objective(study.best_trial)[3]

  rmse_trening = detailed_objective(study.best_trial)[4]
  r2_trening = detailed_objective(study.best_trial)[5]
  mae_trening = detailed_objective(study.best_trial)[6]
  mape_trening = detailed_objective(study.best_trial)[7]

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [9]:
def rmse_validering_StratifiedKFold(modell, datasett, respons, n_splitt):

  skf = StratifiedKFold(n_splits=n_splitt)

  rmse_validering_resultat = []

  for trening, validering in skf.split(datasett, respons):
    X_trening_fold = (datasett.iloc[trening, :]).iloc[:, :-1]
    X_validering_fold = (datasett.iloc[validering, :]).iloc[:, :-1]

    y_trening_fold = (datasett.iloc[trening, :]).iloc[:, -1]
    y_validering_fold = (datasett.iloc[validering, :]).iloc[:, -1]

    modell.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = modell.predict(X_validering_fold)

    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  return mean(rmse_validering)

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
collagen_data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

collagen_data_normal.index = pd.to_datetime(collagen_data_normal.index,
                                            format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [11]:
#Tilfeldighetsfrø
random_seed = 123

# NIR målinger inkludert

## Alternativ 1: Fordelt på enzymtyper

### Alternativ 1: Et testsett

In [21]:
# Velger testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert,\
collagen_test, _, collagen_trening_umarkert,\
_, collagen_trening = trening_testsett_oppdeling_enzym(rå_data,
                                                       collagen_data,
                                                       test_andel=test_andel)

# Deler settene i forklarings- og responsvariabler
X_trening = collagen_trening.iloc[:, :-1]
X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_trening_umarkert = collagen_trening_umarkert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]

y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

# Skalerer treningsdata
skalerer = StandardScaler()
skalerer.fit(X_trening)
X_trening_markert_transformert = skalerer.transform(X_trening_markert)
X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
X_test_transformert = skalerer.transform(X_test)

**CoReg**

Hyperparamter optimalisering med Optuna

In [40]:
def objective(trial):

  parametere = {
      'k1': trial.suggest_int('k1', 1, 20),
      'k2': trial.suggest_int('k2', 1, 20),
      'p1': trial.suggest_int('p1', 1, 20),
      'p2': trial.suggest_int('p2', 1, 20),
      'max_iters': trial.suggest_int('max_iters', 1, 20),
      'pool_size': trial.suggest_int('pool_size', 10, 200)
  }

  coreg = CoReg(**parametere)

  splits = 2
  skf = StratifiedKFold(n_splits = splits)

  rmse_validering_resultat = []

  for train, test in skf.split(collagen_trening_markert,
                               collagen_enzymtyper_markert):

    X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
    skalerer_fold = StandardScaler()
    skalerer_fold.fit(X_trening_fold)

    X_trening_fold_transformert = skalerer_fold.transform(X_trening_fold)
    X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]
    X_validering_fold_transformert = skalerer_fold.transform(X_validering_fold)

    y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
    y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]


    coreg.fit(X_trening_fold_transformert,
              y_trening_fold,
              X_trening_umarkert_transformert)

    y_validering_prediksjon = coreg.predict(X_validering_fold_transformert)
    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=10)

plot_optimization_history(study)

[I 2024-02-22 21:08:54,768] A new study created in memory with name: no-name-96985b2b-e005-4d0f-a7cb-8449ff47052f
[I 2024-02-22 21:10:30,354] Trial 0 finished with value: 4.343334019760488 and parameters: {'k1': 9, 'k2': 17, 'p1': 11, 'p2': 4, 'max_iters': 20, 'pool_size': 148}. Best is trial 0 with value: 4.343334019760488.
[I 2024-02-22 21:12:16,629] Trial 1 finished with value: 4.367730549526704 and parameters: {'k1': 14, 'k2': 12, 'p1': 20, 'p2': 6, 'max_iters': 17, 'pool_size': 177}. Best is trial 0 with value: 4.343334019760488.
[I 2024-02-22 21:13:54,851] Trial 2 finished with value: 4.3615671888382215 and parameters: {'k1': 16, 'k2': 14, 'p1': 6, 'p2': 3, 'max_iters': 18, 'pool_size': 159}. Best is trial 0 with value: 4.343334019760488.
[I 2024-02-22 21:14:08,832] Trial 3 finished with value: 4.321216963222557 and parameters: {'k1': 18, 'k2': 3, 'p1': 4, 'p2': 8, 'max_iters': 20, 'pool_size': 28}. Best is trial 3 with value: 4.321216963222557.
[I 2024-02-22 21:15:08,625] Trial 

Evauleringer av beste modell

In [41]:
# Bruker beste paramtere for studiet for å lage modell
beste_parametere = study.best_params

coreg = CoReg(**beste_parametere)


# Trener modell for videre evaluering ´
coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
y_test_prediksjon = coreg.predict(X_test_transformert)
y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)


# Henter ut evalueringsresulateter av beste modell fra undersøkelse
rmse_test, r2_test, mae_test, mape_test,\
rmse_trening, r2_trening, mae_trening, mape_trening\
= evaluering_beregning(y_test,
                        y_test_prediksjon,
                        y_trening_markert,
                        y_trening_prediksjon)


# Legger resulater til i datasett
resultater = [test_andel,
              rmse_test, r2_test, mae_test, mape_test,
              rmse_trening, r2_trening, mae_trening, mape_trening,
              beste_parametere]


collagen_coreg_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                          "RMSE test",
                                                          "R2 test",
                                                          "MAE test",
                                                          "MAPE test",
                                                          "RMSE trening",
                                                          "R2 trening",
                                                          "MAE trening",
                                                          "MAPE trening",
                                                          "Beste parametere"])

collagen_coreg_resultater_enzym.loc[len(collagen_coreg_resultater_enzym)] = resultater

In [42]:
collagen_coreg_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,3.898984,0.56255,3.156084,0.167436,2.594659,0.73432,2.127169,0.098775,"{'k1': 18, 'k2': 3, 'p1': 4, 'p2': 8, 'max_ite..."


### Alternativ 1: Flere testandeler

In [43]:
collagen_coreg_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                          "RMSE test",
                                                          "R2 test",
                                                          "MAE test",
                                                          "MAPE test",
                                                          "RMSE trening",
                                                          "R2 trening",
                                                          "MAE trening",
                                                          "MAPE trening",
                                                          "Beste parametere"])

test_andeler = [0.1, 0.2]

for test_andel in test_andeler:
  collagen_trening_markert, collagen_enzymtyper_markert,\
  collagen_test, _, collagen_trening_umarkert,\
  _, collagen_trening = trening_testsett_oppdeling_enzym(rå_data,
                                                        collagen_data,
                                                        test_andel=test_andel)

  X_trening = collagen_trening.iloc[:, :-1]
  X_trening_markert = collagen_trening_markert.iloc[:, :-1]
  X_trening_umarkert = collagen_trening_umarkert.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]

  y_trening_markert = collagen_trening_markert.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  def objective(trial):
    parametere = {
      'k1': trial.suggest_int('k1', 1, 20),
      'k2': trial.suggest_int('k2', 1, 20),
      'p1': trial.suggest_int('p1', 1, 20),
      'p2': trial.suggest_int('p2', 1, 20),
      'max_iters': trial.suggest_int('max_iters', 1, 20),
      'pool_size': trial.suggest_int('pool_size', 10, 200)
  }


    coreg = CoReg(**parametere)

    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(collagen_trening_markert, collagen_enzymtyper_markert):
      X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_transformert = skalerer_fold.transform(X_trening_fold)
      X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]
      X_validering_fold_transformert = skalerer_fold.transform(X_validering_fold)

      y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]


      coreg.fit(X_trening_fold_transformert, y_trening_fold, X_trening_umarkert_transformert)
      y_validering_prediksjon = coreg.predict(X_validering_fold_transformert)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  coreg = CoReg(**beste_parametere)


  # Trener modell for videre evaluering ´
  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)


  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                          y_test_prediksjon,
                          y_trening_markert,
                          y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  # Legger resulater til i datasett
  collagen_coreg_resultater_enzym.loc[len(collagen_coreg_resultater_enzym)] = resultater

[I 2024-02-22 21:17:54,140] A new study created in memory with name: no-name-27add118-5862-41a6-8e2f-91a6988f63c9
[I 2024-02-22 21:18:00,012] Trial 0 finished with value: 4.5188423753891955 and parameters: {'k1': 20, 'k2': 6, 'p1': 12, 'p2': 10, 'max_iters': 4, 'pool_size': 23}. Best is trial 0 with value: 4.5188423753891955.
[I 2024-02-22 21:19:52,393] Trial 1 finished with value: 4.273529080256228 and parameters: {'k1': 11, 'k2': 18, 'p1': 3, 'p2': 19, 'max_iters': 11, 'pool_size': 170}. Best is trial 1 with value: 4.273529080256228.
[I 2024-02-22 21:20:11,183] Trial 2 finished with value: 4.317476832445793 and parameters: {'k1': 11, 'k2': 1, 'p1': 17, 'p2': 2, 'max_iters': 6, 'pool_size': 105}. Best is trial 1 with value: 4.273529080256228.
[I 2024-02-22 21:20:31,749] Trial 3 finished with value: 4.49428897400011 and parameters: {'k1': 19, 'k2': 6, 'p1': 20, 'p2': 18, 'max_iters': 17, 'pool_size': 23}. Best is trial 1 with value: 4.273529080256228.
[I 2024-02-22 21:21:22,101] Trial 

In [44]:
collagen_coreg_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,4.202842,0.539360,3.538659,0.163884,3.504764,0.531078,2.838855,0.138456,"{'k1': 13, 'k2': 10, 'p1': 17, 'p2': 2, 'max_i..."
1,0.2,4.231453,0.484766,3.528582,0.182349,3.574843,0.495673,2.944786,0.139037,"{'k1': 11, 'k2': 14, 'p1': 15, 'p2': 16, 'max_..."


## Alternativ 1: Flere treningsandeler

In [45]:
collagen_coreg_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                          "RMSE test",
                                                          "R2 test",
                                                          "MAE test",
                                                          "MAPE test",
                                                          "RMSE trening",
                                                          "R2 trening",
                                                          "MAE trening",
                                                          "MAPE trening",
                                                          "Beste parametere"])


# Setter fast testandel
test_andel = 0.1

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert,\
collagen_test, _, collagen_trening_umarkert,\
_, collagen_trening = trening_testsett_oppdeling_enzym(rå_data,
                                                        collagen_data,
                                                        test_andel=test_andel)
# Treningssett
X_test = collagen_test.iloc[:, :-1]
y_test = collagen_test.iloc[:, -1]

In [46]:
# Andel av treningsettet som skal benyttes til trening av modell
trenings_andeler = [0.8, 0.9, 1.0]

for trenings_andel in trenings_andeler:
  print("Trening:", trenings_andel)
  if trenings_andel < 1.0:
      ubenyttet_andel = (1.0-trenings_andel)
      treningssett_markert, enzymtyper_trening_markert, _, _,\
      treningssett_umarkert, _, treningssett\
      =trening_testsett_oppdeling_enzym(rå_data,
                                        collagen_trening,
                                        test_andel=ubenyttet_andel,
                                        random_seed=random_seed)

  else:
    treningssett_markert = collagen_trening_markert
    enzymtyper_trening_markert = collagen_enzymtyper_markert
    treningssett_umarkert = collagen_trening_umarkert
    treningssett = collagen_trening


  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  def objective(trial):

    parametere = {
        'k1': trial.suggest_int('k1', 1, 20),
        'k2': trial.suggest_int('k2', 1, 20),
        'p1': trial.suggest_int('p1', 1, 20),
        'p2': trial.suggest_int('p2', 1, 20),
        'max_iters': trial.suggest_int('max_iters', 1, 20),
        'pool_size': trial.suggest_int('pool_size', 10, 200)
    }


    coreg = CoReg(**parametere)

    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(collagen_trening_markert, collagen_enzymtyper_markert):
      X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_transformert = skalerer_fold.transform(X_trening_fold)
      X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]
      X_validering_fold_transformert = skalerer_fold.transform(X_validering_fold)

      y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]


      coreg.fit(X_trening_fold_transformert, y_trening_fold, X_trening_umarkert_transformert)
      y_validering_prediksjon = coreg.predict(X_validering_fold_transformert)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  coreg = CoReg(**beste_parametere)


  # Trener modell for videre evaluering ´
  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)


  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                          y_test_prediksjon,
                          y_trening_markert,
                          y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  # Legger resulater til i datasett
  collagen_coreg_resultater_enzym.loc[len(collagen_coreg_resultater_enzym)] = resultater

[I 2024-02-22 21:27:46,220] A new study created in memory with name: no-name-cd5bdb60-49c7-4a1e-b3df-1cbfc41631a4


Trening: 0.8


[I 2024-02-22 21:28:17,387] Trial 0 finished with value: 4.274516385552471 and parameters: {'k1': 9, 'k2': 3, 'p1': 13, 'p2': 20, 'max_iters': 11, 'pool_size': 99}. Best is trial 0 with value: 4.274516385552471.
[I 2024-02-22 21:28:21,741] Trial 1 finished with value: 4.4085159921588755 and parameters: {'k1': 18, 'k2': 2, 'p1': 18, 'p2': 10, 'max_iters': 9, 'pool_size': 11}. Best is trial 0 with value: 4.274516385552471.
[I 2024-02-22 21:28:27,747] Trial 2 finished with value: 4.3548756772826716 and parameters: {'k1': 10, 'k2': 18, 'p1': 16, 'p2': 3, 'max_iters': 2, 'pool_size': 48}. Best is trial 0 with value: 4.274516385552471.
[I 2024-02-22 21:28:31,383] Trial 3 finished with value: 4.46405360886772 and parameters: {'k1': 1, 'k2': 16, 'p1': 4, 'p2': 7, 'max_iters': 9, 'pool_size': 13}. Best is trial 0 with value: 4.274516385552471.
[I 2024-02-22 21:28:50,442] Trial 4 finished with value: 4.611791481926012 and parameters: {'k1': 1, 'k2': 7, 'p1': 3, 'p2': 10, 'max_iters': 12, 'pool_s

Trening: 0.9


[I 2024-02-22 21:31:41,625] Trial 0 finished with value: 4.330667174796332 and parameters: {'k1': 2, 'k2': 11, 'p1': 12, 'p2': 14, 'max_iters': 1, 'pool_size': 107}. Best is trial 0 with value: 4.330667174796332.
[I 2024-02-22 21:31:56,180] Trial 1 finished with value: 4.306756857304609 and parameters: {'k1': 8, 'k2': 8, 'p1': 19, 'p2': 14, 'max_iters': 7, 'pool_size': 56}. Best is trial 1 with value: 4.306756857304609.
[I 2024-02-22 21:34:37,566] Trial 2 finished with value: 4.092505401273117 and parameters: {'k1': 20, 'k2': 3, 'p1': 9, 'p2': 20, 'max_iters': 17, 'pool_size': 186}. Best is trial 2 with value: 4.092505401273117.
[I 2024-02-22 21:34:42,513] Trial 3 finished with value: 4.509315820000308 and parameters: {'k1': 1, 'k2': 20, 'p1': 4, 'p2': 12, 'max_iters': 9, 'pool_size': 10}. Best is trial 2 with value: 4.092505401273117.
[I 2024-02-22 21:35:28,195] Trial 4 finished with value: 4.353904020937014 and parameters: {'k1': 14, 'k2': 13, 'p1': 16, 'p2': 12, 'max_iters': 9, 'poo

Trening: 1.0


[I 2024-02-22 21:42:42,352] Trial 0 finished with value: 4.283112247848642 and parameters: {'k1': 16, 'k2': 11, 'p1': 11, 'p2': 5, 'max_iters': 12, 'pool_size': 161}. Best is trial 0 with value: 4.283112247848642.
[I 2024-02-22 21:43:44,257] Trial 1 finished with value: 4.402250111617325 and parameters: {'k1': 20, 'k2': 14, 'p1': 6, 'p2': 19, 'max_iters': 14, 'pool_size': 62}. Best is trial 0 with value: 4.283112247848642.
[I 2024-02-22 21:43:48,804] Trial 2 finished with value: 4.690304782548122 and parameters: {'k1': 20, 'k2': 17, 'p1': 20, 'p2': 11, 'max_iters': 3, 'pool_size': 11}. Best is trial 0 with value: 4.283112247848642.
[I 2024-02-22 21:43:49,520] Trial 3 finished with value: 4.4873661022536115 and parameters: {'k1': 6, 'k2': 14, 'p1': 4, 'p2': 17, 'max_iters': 1, 'pool_size': 15}. Best is trial 0 with value: 4.283112247848642.
[I 2024-02-22 21:44:41,364] Trial 4 finished with value: 4.2326731324936535 and parameters: {'k1': 13, 'k2': 6, 'p1': 8, 'p2': 10, 'max_iters': 8, '

In [48]:
collagen_coreg_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,4.202842,0.539360,3.538659,0.163884,3.504764,0.531078,2.838855,0.138456,"{'k1': 13, 'k2': 10, 'p1': 17, 'p2': 2, 'max_i..."
1,0.2,4.231453,0.484766,3.528582,0.182349,3.574843,0.495673,2.944786,0.139037,"{'k1': 11, 'k2': 14, 'p1': 15, 'p2': 16, 'max_..."
2,0.8,4.279046,0.522504,3.700046,0.171025,2.074572,0.839583,1.689005,0.079468,"{'k1': 2, 'k2': 8, 'p1': 2, 'p2': 20, 'max_ite..."
3,0.9,4.181782,0.543964,3.511469,0.168410,2.647131,0.730794,2.150712,0.101770,"{'k1': 20, 'k2': 3, 'p1': 9, 'p2': 20, 'max_it..."
4,1.0,4.010906,0.580472,3.344239,0.155332,3.220919,0.603956,2.638810,0.126607,"{'k1': 13, 'k2': 6, 'p1': 8, 'p2': 10, 'max_it..."


## Alternativ 2: Fordelt på dag og kontinuitet

### Alternativ 2: Et testsett

In [ ]:
# Valg av testandel
test_andel = 0.2

# Oppdeling av datasett til trenin og testsett, samt markert og umarkert del
collagen_trening_markert, collagen_test, collagen_trening_umarkert, collagen_trening = trening_testsett_oppdeling_dag(rå_data,
                                                                                                                      collagen_data,
                                                                                                                      test_andel)
# Forklarings- og responsvariabler
X_trening = collagen_trening.iloc[:, :-1]
X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_trening_umarkert = collagen_trening_umarkert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]

y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

# Skalering
skalerer = StandardScaler()
skalerer.fit(X_trening)
X_trening_markert_transformert = skalerer.transform(X_trening_markert)
X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
X_test_transformert = skalerer.transform(X_test)

<ipython-input-8-fc5ecd67e2cb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert['RawMaterialMix'] = råmateriale_data


**CoReg**

Hyperparamter optimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  rmse_test = rmse(y_test, y_test_prediksjon)
  return rmse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=10)

plot_optimization_history(study)

[I 2024-02-15 14:28:53,433] A new study created in memory with name: no-name-4a0c3411-e2ea-4af7-95d2-e672050f4c97
[I 2024-02-15 14:30:28,930] Trial 0 finished with value: 3.946188071342318 and parameters: {'k1': 9, 'k2': 3, 'p1': 9, 'p2': 10}. Best is trial 0 with value: 3.946188071342318.
[I 2024-02-15 14:32:09,881] Trial 1 finished with value: 3.8542007899294615 and parameters: {'k1': 4, 'k2': 1, 'p1': 1, 'p2': 1}. Best is trial 1 with value: 3.8542007899294615.
[I 2024-02-15 14:33:46,219] Trial 2 finished with value: 3.7641835996585167 and parameters: {'k1': 8, 'k2': 3, 'p1': 3, 'p2': 10}. Best is trial 2 with value: 3.7641835996585167.
[I 2024-02-15 14:35:20,185] Trial 3 finished with value: 3.670252371335196 and parameters: {'k1': 9, 'k2': 8, 'p1': 6, 'p2': 2}. Best is trial 3 with value: 3.670252371335196.
[I 2024-02-15 14:36:56,081] Trial 4 finished with value: 3.6237016038207743 and parameters: {'k1': 6, 'k2': 2, 'p1': 2, 'p2': 9}. Best is trial 4 with value: 3.6237016038207743

Evauleringer av beste modell

In [ ]:
def detailed_objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)


  rmse_trening = rmse(y_trening_markert, y_trening_prediksjon)
  r2_trening = r2_score(y_trening_markert, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening_markert, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening_markert, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

rmse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mae_test_resultat = detailed_objective(study.best_trial)[2]
mape_test_resultat = detailed_objective(study.best_trial)[3]

rmse_trening_resultat = detailed_objective(study.best_trial)[4]
r2_trening_resultat = detailed_objective(study.best_trial)[5]
mae_trening_resultat = detailed_objective(study.best_trial)[6]
mape_trening_resultat = detailed_objective(study.best_trial)[7]

collagen_coreg_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                          "RMSE test",
                                                          "R2 test",
                                                          "MAE test",
                                                          "MAPE test",
                                                          "RMSE trening",
                                                          "R2 trening",
                                                          "MAE trening",
                                                          "MAPE trening",
                                                          "Beste parametere"])

beste_parametere = study.best_params

resultater = [test_andel,
              rmse_test_resultat,
              r2_test_resultat,
              mae_test_resultat,
              mape_test_resultat,
              rmse_trening_resultat,
              r2_trening_resultat,
              mae_trening_resultat,
              mape_trening_resultat,
              beste_parametere]

collagen_coreg_resultater_enzym.loc[len(collagen_coreg_resultater_enzym)] = resultater
print(collagen_coreg_resultater_enzym)

   Test andel  RMSE test   R2 test  MAE test  MAPE test  RMSE trening  \
0         0.2   3.654918  0.453441  3.058378   0.146384      1.390003   

   R2 trening  MAE trening  MAPE trening                      Beste parametere  
0    0.925645     0.997047      0.044034  {'k1': 8, 'k2': 2, 'p1': 3, 'p2': 7}  


### Alternativ 2: Flere testandeler

In [ ]:
collagen_coreg_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

test_andeler = [0.1, 0.2, 0.3, 0.4, 0.5]

for test_andel in test_andeler:
  collagen_trening_markert, collagen_test, collagen_trening_umarkert, collagen_trening = trening_testsett_oppdeling_dag(rå_data,
                                                                                                                      collagen_data,
                                                                                                                      test_andel)


  X_trening = collagen_trening.iloc[:, :-1]
  X_trening_markert = collagen_trening_markert.iloc[:, :-1]
  X_trening_umarkert = collagen_trening_umarkert.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]

  y_trening_markert = collagen_trening_markert.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  def objective(trial):
    parametere = {
        'k1': trial.suggest_int('k1', 1, 10),
        'k2': trial.suggest_int('k2', 1, 10),
        'p1': trial.suggest_int('p1', 1, 10),
        'p2': trial.suggest_int('p2', 1, 10)
    }

    coreg = CoReg()

    coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
    y_test_prediksjon = coreg.predict(X_test_transformert)
    rmse_test = rmse(y_test, y_test_prediksjon)
    return rmse_test

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)


  def detailed_objective(trial):
    parametere = {
        'k1': trial.suggest_int('k1', 1, 10),
        'k2': trial.suggest_int('k2', 1, 10),
        'p1': trial.suggest_int('p1', 1, 10),
        'p2': trial.suggest_int('p2', 1, 10)
    }

    coreg = CoReg()

    coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
    y_test_prediksjon = coreg.predict(X_test_transformert)
    y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

    rmse_test = rmse(y_test, y_test_prediksjon)
    r2_test = r2_score(y_test, y_test_prediksjon)
    mae_test = mean_absolute_error(y_test, y_test_prediksjon)
    mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)


    rmse_trening = rmse(y_trening_markert, y_trening_prediksjon)
    r2_trening = r2_score(y_trening_markert, y_trening_prediksjon)
    mae_trening = mean_absolute_error(y_trening_markert, y_trening_prediksjon)
    mape_trening = mean_absolute_percentage_error(y_trening_markert, y_trening_prediksjon)

    return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

  rmse_test_resultat = detailed_objective(study.best_trial)[0]
  r2_test_resultat = detailed_objective(study.best_trial)[1]
  mae_test_resultat = detailed_objective(study.best_trial)[2]
  mape_test_resultat = detailed_objective(study.best_trial)[3]

  rmse_trening_resultat = detailed_objective(study.best_trial)[4]
  r2_trening_resultat = detailed_objective(study.best_trial)[5]
  mae_trening_resultat = detailed_objective(study.best_trial)[6]
  mape_trening_resultat = detailed_objective(study.best_trial)[7]

  beste_parametere = study.best_params

  resultater = [test_andel,
                rmse_test_resultat,
                r2_test_resultat,
                mae_test_resultat,
                mape_test_resultat,
                rmse_trening_resultat,
                r2_trening_resultat,
                mae_trening_resultat,
                mape_trening_resultat,
                beste_parametere]

  collagen_coreg_resultater_dag.loc[len(collagen_coreg_resultater_dag)] = resultater

<ipython-input-8-fc5ecd67e2cb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert['RawMaterialMix'] = råmateriale_data
[I 2024-02-16 12:02:10,534] A new study created in memory with name: no-name-d74b9daf-55c3-44f0-b2a5-598063eac103
[I 2024-02-16 12:03:36,415] Trial 0 finished with value: 3.8877537424688855 and parameters: {'k1': 3, 'k2': 8, 'p1': 1, 'p2': 9}. Best is trial 0 with value: 3.8877537424688855.
[I 2024-02-16 12:05:00,324] Trial 1 finished with value: 4.3391025260368075 and parameters: {'k1': 8, 'k2': 10, 'p1': 2, 'p2': 8}. Best is trial 0 with value: 3.8877537424688855.
[I 2024-02-16 12:06:24,139] Trial 2 finished with value: 3.287827978347458 and parameters: {'k1': 7, 'k2': 9, 'p1': 9, 'p2': 3}. Best is trial 2 with valu

In [ ]:
collagen_coreg_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,3.732778,0.105147,3.265388,0.162898,1.462619,0.927568,1.056689,0.046517,"{'k1': 7, 'k2': 9, 'p1': 9, 'p2': 3}"
1,0.2,3.516569,0.417800,3.157804,0.147428,1.351680,0.926205,1.045626,0.046103,"{'k1': 6, 'k2': 5, 'p1': 6, 'p2': 1}"
2,0.3,4.132328,0.045528,3.398061,0.168199,1.339703,0.932920,1.020832,0.044069,"{'k1': 9, 'k2': 6, 'p1': 6, 'p2': 1}"
3,0.4,3.490240,0.540573,2.805247,0.124713,1.446574,0.931633,1.177557,0.053503,"{'k1': 7, 'k2': 8, 'p1': 2, 'p2': 8}"
4,0.5,3.917921,0.475110,2.899328,0.148522,1.398529,0.908545,1.139016,0.045939,"{'k1': 3, 'k2': 9, 'p1': 5, 'p2': 10}"
